In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from MLP import MLP

In [2]:
train_set=torchvision.datasets.CIFAR10(root="../data",train=True,transform=transforms.ToTensor(),download=False)
test_set=torchvision.datasets.CIFAR10(root="../data",train=False,transform=transforms.ToTensor(),download=False)

In [3]:
batch_size=64
lr=0.01
epoch=20
writer=SummaryWriter("logs")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
trainLoader=DataLoader(train_set,batch_size=batch_size,shuffle=True,num_workers=0,drop_last=False)
testLoader=DataLoader(test_set,batch_size=batch_size,shuffle=True,num_workers=0,drop_last=False)

In [4]:
model=MLP(3072)
loss=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=lr)

In [5]:
def train(trainLoader,model,loss_fn,optimizer):
    print(f"training on:{device}")
    model=model.to(device)
    size=len(trainLoader.dataset)
    
    for batch,(X,y) in enumerate(trainLoader):
        X=X.to(device)
        y=y.to(device)
        pred=model(X)
        loss=loss_fn(pred,y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch%10==0:
            loss,current=loss.item(),batch*len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")

In [6]:
max_correct=0
def test(testLoader, model, loss_fn,max_correct):
    model = model.to(device)
    size = len(testLoader.dataset)
    num_batches = len(testLoader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in testLoader:
            X = X.to(device)
            y = y.to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches

    correct /= size
    
    if correct>max_correct:
        torch.save(model.state_dict(),'mlp.params')
        max_correct=correct

    print(f"test error:\n accuracy:{(100 * correct):>0.1f}%,avg loss: {test_loss:>8f} \n")
    return max_correct,correct

In [7]:
for t in range(epoch):
    print(f"epoch {t+1} \n----------------------------------------------")
    train(trainLoader,model,loss,optimizer)
    max_correct,correct=test(testLoader,model,loss,max_correct)
    writer.add_scalar("accuracy",correct,t)
    
writer.close()

epoch 1 
----------------------------------------------
training on:cuda
loss:2.304476 [    0/50000]
loss:2.304917 [  640/50000]
loss:2.284623 [ 1280/50000]
loss:2.283258 [ 1920/50000]
loss:2.282411 [ 2560/50000]
loss:2.288828 [ 3200/50000]
loss:2.296762 [ 3840/50000]
loss:2.292325 [ 4480/50000]
loss:2.275413 [ 5120/50000]
loss:2.296201 [ 5760/50000]
loss:2.295669 [ 6400/50000]
loss:2.271317 [ 7040/50000]
loss:2.261642 [ 7680/50000]
loss:2.270866 [ 8320/50000]
loss:2.256522 [ 8960/50000]
loss:2.243615 [ 9600/50000]
loss:2.222149 [10240/50000]
loss:2.249742 [10880/50000]
loss:2.223388 [11520/50000]
loss:2.242452 [12160/50000]
loss:2.261486 [12800/50000]
loss:2.233497 [13440/50000]
loss:2.195861 [14080/50000]
loss:2.261210 [14720/50000]
loss:2.177195 [15360/50000]
loss:2.225741 [16000/50000]
loss:2.180796 [16640/50000]
loss:2.224156 [17280/50000]
loss:2.227991 [17920/50000]
loss:2.139794 [18560/50000]
loss:2.184254 [19200/50000]
loss:2.177902 [19840/50000]
loss:2.148236 [20480/50000]
los